In [1]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.8 MB/s eta 0:00:00


In [2]:
!pip install python-louvain

Imports

In [3]:
import pandas as pd
import zipfile
import io
import pickle
from scipy.stats import spearmanr
import pycountry
import requests
from matplotlib import cm
import community as community_louvain
from datetime import datetime, timedelta
from tqdm import tqdm
import plotly.express as px
import os
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import glob
import plotly.graph_objects as go
from google.colab import drive
import dask.dataframe as dd
import community as community_louvain
import community.community_louvain as community_louvain
drive.mount('/content/drive')
pd.set_option('display.max_columns', None)  # Show all columns
# pd.reset_option('display.max_columns')

Mounted at /content/drive


###Global:

In [4]:
# url = "https://www.gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt"
# headers = requests.get(url).text.strip().split('\t')
# headers

In [5]:
#fixing columns mismatch - there is one attribute missing 3 times (Actor1Geo_ADM2Code., Actor2Geo_ADM2Code.,ActionGeo_ADM2Code.)
headers = ['GLOBALEVENTID',
 'SQLDATE',
 'MonthYear',
 'Year',
 'FractionDate',
 'Actor1Code',
 'Actor1Name',
 'Actor1CountryCode',
 'Actor1KnownGroupCode',
 'Actor1EthnicCode',
 'Actor1Religion1Code',
 'Actor1Religion2Code',
 'Actor1Type1Code',
 'Actor1Type2Code',
 'Actor1Type3Code',
 'Actor2Code',
 'Actor2Name',
 'Actor2CountryCode',
 'Actor2KnownGroupCode',
 'Actor2EthnicCode',
 'Actor2Religion1Code',
 'Actor2Religion2Code',
 'Actor2Type1Code',
 'Actor2Type2Code',
 'Actor2Type3Code',
 'IsRootEvent',
 'EventCode',
 'EventBaseCode',
 'EventRootCode',
 'QuadClass',
 'GoldsteinScale',
 'NumMentions',
 'NumSources',
 'NumArticles',
 'AvgTone',
 'Actor1Geo_Type',
 'Actor1Geo_FullName',
 'Actor1Geo_CountryCode',
 'Actor1Geo_ADM1Code',
 'Actor1Geo_ADM2Code',
 'Actor1Geo_Lat',
 'Actor1Geo_Long',
 'Actor1Geo_FeatureID',
 'Actor2Geo_Type',
 'Actor2Geo_FullName',
 'Actor2Geo_CountryCode',
 'Actor2Geo_ADM1Code',
 'Actor2Geo_ADM2Code',
 'Actor2Geo_Lat',
 'Actor2Geo_Long',
 'Actor2Geo_FeatureID',
 'ActionGeo_Type',
 'ActionGeo_FullName',
 'ActionGeo_CountryCode',
 'ActionGeo_ADM1Code',
 'ActionGeo_ADM2Code',
 'ActionGeo_Lat',
 'ActionGeo_Long',
 'ActionGeo_FeatureID',
 'DATEADDED',
 'SOURCEURL']

In [6]:
# Israeli name variants
ISRAEL_VARIANTS = set([
    'israel', 'the state of israel', 'state of israel',
    'israeli', 'israeli government', 'israeli military',
    'jerusalem', 'tel aviv', 'isr', 'idf'])

In [7]:
signif_columns = [
 'SQLDATE',
 'Actor1Code',
 'Actor1Name',
 'Actor1CountryCode',
 'Actor2Code',
 'Actor2Name',
 'Actor2CountryCode',
 'IsRootEvent',
 'EventCode',
 'EventBaseCode',
 'EventRootCode',
 'QuadClass',
 'GoldsteinScale',
 'NumMentions',
 'NumSources',
 'NumArticles',
 'AvgTone',
 'ActionGeo_FullName',
 'ActionGeo_CountryCode',
 'DATEADDED',
 'SOURCEURL']

## Data collection and preprocessing

In [ ]:
#Extracting GDELT's raw files links
url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
response = requests.get(url, timeout=10)
response.raise_for_status()

HTTPError: 403 Client Error: Forbidden for url: http://data.gdeltproject.org/gdeltv2/masterfilelist.txt

In [ ]:
#Filtering according to dates:
#Set date range
start_date = datetime(2023, 10, 7)
end_date = datetime.today()

filtered_urls = []
bad_lines = []
lines = response.text.splitlines()
for line in lines:
    parts = line.strip().split()
    if len(parts) != 3:
        continue

    url = parts[2]
    try:
        # Extract timestamp from filename
        filename = url.split("/")[-1]
        timestamp_str = filename.split(".")[0]  # e.g. 20231007120000
        file_datetime = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")

        # Check if in range
        if start_date <= file_datetime <= end_date and url.endswith("export.CSV.zip"):
            filtered_urls.append(url)
    except Exception:
      print(f"Problem with parsing url: {url}")
len(filtered_urls)

58976

In [ ]:
output_dir = "/content/drive/MyDrive/Miki/GDELT2_Data_post_7th"

Mounted at /content/drive


In [ ]:
processed_log = "processed.txt"
processed_urls = set()

if os.path.exists(processed_log):
    with open(processed_log, "r") as f:
        processed_urls = set(line.strip() for line in f if line.strip())
for file_url in tqdm(filtered_urls, desc="Processing files"):
    if file_url in processed_urls:
        continue  # Skip already processed
    try:

        r = requests.get(file_url, timeout=30)
        r.raise_for_status()

        # Unzip the content in memory
        with zipfile.ZipFile(io.BytesIO(r.content)) as z:
            for filename in z.namelist():
                with z.open(filename) as f:
                    df = pd.read_csv(f,names = headers, sep='\t', encoding='ISO-8859-1', dtype=str,low_memory=False)

                    ts = file_url.split("/")[-1].split(".")[0]
                    df.to_parquet(f"{output_dir}/{ts}.parquet")
                    # Log success
                    with open(processed_log, "a") as logf:
                        logf.write(file_url + "\n")

    except Exception as e:
        print(f"Failed to process {file_url}: {e}")


Processing files:  85%|████████▍ | 50087/58976 [5:37:19<4:40:19,  1.89s/it]

Failed to process http://data.gdeltproject.org/gdeltv2/20250311194500.export.CSV.zip: 500 Server Error: Internal Server Error for url: http://data.gdeltproject.org/gdeltv2/20250311194500.export.CSV.zip


Processing files:  85%|████████▍ | 50091/58976 [5:37:25<5:32:13,  2.24s/it]

Failed to process http://data.gdeltproject.org/gdeltv2/20250311204500.export.CSV.zip: 500 Server Error: Internal Server Error for url: http://data.gdeltproject.org/gdeltv2/20250311204500.export.CSV.zip


Processing files:  85%|████████▍ | 50093/58976 [5:37:33<8:00:58,  3.25s/it]

...

Processing files:  88%|████████▊ | 51606/58976 [6:29:34<4:14:06,  2.07s/it]

Failed to process http://data.gdeltproject.org/gdeltv2/20250327154500.export.CSV.zip: 500 Server Error: Internal Server Error for url: http://data.gdeltproject.org/gdeltv2/20250327154500.export.CSV.zip


Processing files:  88%|████████▊ | 51884/58976 [6:32:13<3:28:25,  1.76s/it]

Failed to process http://data.gdeltproject.org/gdeltv2/20250330131500.export.CSV.zip: 500 Server Error: Internal Server Error for url: http://data.gdeltproject.org/gdeltv2/20250330131500.export.CSV.zip


Processing files: 100%|██████████| 58976/58976 [7:18:46<00:00,  2.24it/s]


### Preprocessing:

In [ ]:
input_folder = '/content/drive/MyDrive/Miki/GDELT2_Data_post_7th'
output_folder = '/content/drive/MyDrive/Miki/GDELT2_Data_post_7th/Preprocessed/Israeli_Interactions'
input_list_file = os.path.join(output_folder, 'input_files.txt')
processed_log = os.path.join(output_folder, 'processed_files.txt')

os.makedirs(output_folder, exist_ok=True)

BATCH_SIZE = 100

#Create input file list if missing
if not os.path.exists(input_list_file):
    print("Creating static input list...")
    parquet_files = sorted([
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.endswith('.parquet')
    ])
    with open(input_list_file, 'w') as f:
        for path in parquet_files:
            f.write(path + '\n')

# Read input + checkpoint files
with open(input_list_file, 'r') as f:
    parquet_files = [line.strip() for line in f]

if os.path.exists(processed_log):
    with open(processed_log, 'r') as f:
        processed_files = set(f.read().splitlines())
else:
    processed_files = set()

# Process files with filtering
file_buffer = []
file_counter = 0
output_counter = 0

for file_path in tqdm(parquet_files, desc="Preprocessing GDELT Files"):
    filename = os.path.basename(file_path)

    if filename in processed_files:
        continue  # Already processed

    try:
        df = pd.read_parquet(file_path, columns=signif_columns)
        filtered_df = df[
            df['Actor1Name'].str.lower().isin(ISRAEL_VARIANTS) |
            df['Actor2Name'].str.lower().isin(ISRAEL_VARIANTS) | df['Actor1CountryCode'].str.lower().isin(ISRAEL_VARIANTS) | df['Actor2CountryCode'].str.lower().isin(ISRAEL_VARIANTS) |df['ActionGeo_CountryCode'].str.lower().isin(ISRAEL_VARIANTS) | df['ActionGeo_FullName'].str.lower().isin(ISRAEL_VARIANTS)]

        if not filtered_df.empty:
            file_buffer.append(filtered_df)
            file_counter += 1

        # Mark file as processed
        with open(processed_log, 'a') as f:
            f.write(filename + '\n')
        processed_files.add(filename)

        # Save in batches
        if file_counter >= BATCH_SIZE:
            out_path = os.path.join(output_folder, f'israel_filtered_batch_{output_counter}.parquet')
            pd.concat(file_buffer, ignore_index=True).to_parquet(out_path)
            file_buffer = []
            file_counter = 0
            output_counter += 1

    except Exception as e:
        print(f"[ERROR] Failed processing {filename}: {e}")

# === Save remaining records ===
if file_buffer:
    out_path = os.path.join(output_folder, f'israel_filtered_batch_{output_counter}.parquet')
    pd.concat(file_buffer, ignore_index=True).to_parquet(out_path)


Creating static input list...


Preprocessing GDELT Files: 100%|██████████| 58825/58825 [8:31:10<00:00,  1.92it/s]


Merging all parquets to a single one:

In [ ]:
output_dir = '/content/drive/MyDrive/Miki/GDELT2_Data_post_7th/Preprocessed/Israeli_Interactions'
parquet_files = glob.glob(os.path.join(output_dir, '*.parquet'))
list_dfs = [pd.read_parquet(f) for f in parquet_files]
merged_df = pd.concat(list_dfs, ignore_index=True)

# Define the path for the output merged parquet file
output_parquet_path = os.path.join(output_dir, 'merged_israeli_interactions.parquet')

# Save the merged DataFrame to a single parquet file
merged_df.to_parquet(output_parquet_path)

print(f"Merged {len(parquet_files)} parquet files into {output_parquet_path}")
print(f"Shape of the merged DataFrame: {merged_df.shape}")
total_memory = merged_df.memory_usage(deep=True).sum()
print(f"Total memory usage: {total_memory / (1024 ** 2):.2f} MB")

Merged 588 parquet files into /content/drive/MyDrive/Miki/GDELT2_Data_post_7th/Preprocessed/Israeli_Interactions/merged_israeli_interactions.parquet
Shape of the merged DataFrame: (4815024, 21)
Total memory usage: 6359.54 MB


In [ ]:
merged_df

,SQLDATE,Actor1Code,Actor1Name,Actor1CountryCode,Actor2Code,Actor2Name,Actor2CountryCode,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_CountryCode,DATEADDED,SOURCEURL
0,20231007,None,None,None,ISRSET,ISRAEL,ISR,1,190,190,19,4,-10.0,12,6,12,-11.2273361227336,Israel,IS,20231007000000,https://www.iranherald.com/news/273991510/pale...
1,20231007,ISR,ISRAEL,ISR,None,None,None,0,129,129,12,3,-5.0,5,1,5,-0.81234768480909,"Kyiv, Kyyiv, Misto, Ukraine",UP,20231007000000,https://www.politico.com/newsletters/national-...
2,20231007,ISR,ISRAEL,ISR,BUS,PRODUCER,None,0,129,129,12,3,-5.0,5,1,5,-0.81234768480909,"Kyiv, Kyyiv, Misto, Ukraine",UP,20231007000000,https://www.politico.com/newsletters/national-...
3,20231007,ISR,ISRAEL,ISR,ISRSET,ISRAEL,ISR,1,193,193,19,4,-10.0,4,1,4,-10.8786610878661,Israel,IS,20231007000000,https://www.bignewsnetwork.com/news/273991510/...
4,20231007,ISRSET,ISRAEL,ISR,ISR,ISRAEL,ISR,1,182,182,18,4,-9.5,12,6,12,-11.2273361227336,Israel,IS,20231007000000,https://www.iranherald.com/news/273991510/pale...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4815019,20250612,UAF,MILITANT,None,ISR,ISRAELI,ISR,0,015,015,01,1,0.0,10,1,10,-6.57193605683837,"Gaza, Israel (general), Israel",IS,20250612111500,https://www.yahoo.com/news/humanitarian-worker...
4815020,20250612,USA,UNITED STATES,USA,ISR,ISRAEL,ISR,0,114,114,11,3,-2.0,4,2,4,-9.63518192603414,United States,US,20250612111500,https://www.yahoo.com/news/contributor-scars-u...
4815021,20250612,USAGOV,THE US,USA,ISR,TEL AVIV,ISR,1,172,172,17,4,-5.0,1,1,1,-6.60660660660661,"Tehran, Tehran, Iran",IR,20250612111500,https://www.aa.com.tr/en/middle-east/us-embass...
4815022,20250612,USAGOV,THE US,USA,ISR,TEL AVIV,ISR,1,172,172,17,4,-5.0,1,1,1,-6.60660660660661,"Washington, District of Columbia, United States",US,20250612111500,https://www.aa.com.tr/en/middle-east/us-embass...


In [ ]:
cameo_code_path = "/content/drive/MyDrive/Miki/cameocodes.txt"
with open(cameo_code_path, 'r') as f:
  cameo_codes_content = f.readlines()
cameo_codes_content = [line.strip() for line in cameo_codes_content]

In [ ]:
data = []
for line in cameo_codes_content[1:]:
    parts = line.split('\t' if '\t' in line else '\\')
    if len(parts) == 2:
        code = parts[0].strip()
        description = parts[1].strip()
        data.append((code, description))
    elif len(parts) == 1:
        data.append((parts[0].strip(), ''))
df_cameo_codes = pd.DataFrame(data, columns=['CAMEO_Code', 'Description'])

In [ ]:
df_cameo_codes['CAMEO_Code'] = df_cameo_codes['CAMEO_Code'].astype(str)
df_cameo_codes.to_csv("/content/drive/MyDrive/Miki/cameocodes.csv", index=False)

In [ ]:
cameo_codes_list = list(df_cameo_codes['CAMEO_Code'].unique())

## Joining with CAMEO event descriptions find online

In [3]:
path = '/content/drive/MyDrive/Miki/GDELT2_Data_post_7th/Preprocessed/Israeli_Interactions/merged_israeli_interactions.parquet'
df = dd.read_parquet(path)

NameError: name 'dd' is not defined

In [ ]:
cameo_df = pd.read_csv('/content/drive/MyDrive/Miki/cameocodes.csv', dtype={'CAMEO_Code': str})
cameo_df.rename(columns={'CAMEO_Code': 'EventBaseCode','Description':'Event Description'}, inplace=True)
cameo_df

,EventBaseCode,Event Description
0,01,MAKE PUBLIC STATEMENT
1,010,"Make statement, not specified below"
2,011,Decline comment
3,012,Make pessimistic comment
4,013,Make optimistic comment
...,...,...
305,202,Engage in mass killings
306,203,Engage in ethnic cleansing
307,204,"Use weapons of mass destruction, not specified..."
308,2041,"Use chemical, biological, or radiologicalweapons"


In [ ]:
df.head()

,SQLDATE,Actor1Code,Actor1Name,Actor1CountryCode,Actor2Code,Actor2Name,Actor2CountryCode,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,ActionGeo_FullName,ActionGeo_CountryCode,DATEADDED,SOURCEURL
0,20231007,<NA>,<NA>,<NA>,ISRSET,ISRAEL,ISR,1,190,190,19,4,-10.0,12,6,12,-11.2273361227336,Israel,IS,20231007000000,https://www.iranherald.com/news/273991510/pale...
1,20231007,ISR,ISRAEL,ISR,<NA>,<NA>,<NA>,0,129,129,12,3,-5.0,5,1,5,-0.81234768480909,"Kyiv, Kyyiv, Misto, Ukraine",UP,20231007000000,https://www.politico.com/newsletters/national-...
2,20231007,ISR,ISRAEL,ISR,BUS,PRODUCER,<NA>,0,129,129,12,3,-5.0,5,1,5,-0.81234768480909,"Kyiv, Kyyiv, Misto, Ukraine",UP,20231007000000,https://www.politico.com/newsletters/national-...
3,20231007,ISR,ISRAEL,ISR,ISRSET,ISRAEL,ISR,1,193,193,19,4,-10.0,4,1,4,-10.8786610878661,Israel,IS,20231007000000,https://www.bignewsnetwork.com/news/273991510/...
4,20231007,ISRSET,ISRAEL,ISR,ISR,ISRAEL,ISR,1,182,182,18,4,-9.5,12,6,12,-11.2273361227336,Israel,IS,20231007000000,https://www.iranherald.com/news/273991510/pale...


In [ ]:
# Function to join each partition
def join_partition(partition, small_df):
    result = partition.merge(cameo_df, how='left', on='EventBaseCode')
    return result
# Infer meta (required by Dask to know output schema)
meta = join_partition(df._meta, cameo_df)
# Perform broadcast join
joined_ddf = df.map_partitions(join_partition, cameo_df, meta=meta)

joined_ddf = joined_ddf.repartition(npartitions=1)
joined_ddf.to_parquet('/content/drive/MyDrive/Miki/israeli_interactions_post_7th_w_cameo.parquet', overwrite=True)